In [29]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import typing
import time

In [26]:
os.listdir("./data/BLAST/")

['NC_003861.html',
 'OK362288.html',
 'OK377341.html',
 'OK377342.html',
 'OK377343.html',
 'OK424595.html']

In [21]:
contents = {}

for file in os.listdir("./data/BLAST/"):
    with open("./data/BLAST/" + file, 'r') as fp:
        contents[file.replace(".html", "")] = BeautifulSoup(fp.read(), "html.parser")

In [33]:
contents["NC_003861"].find("table", attrs = {"id": "dscTable"});

In [34]:
# a typical table record

"""

<tr id="dtr_61657622" ind="7" class="odd dflLnk">
<td class="l c0"><span class="ind">7</span><input type="checkbox" id="chk_7" class="cb" name="getSeqGi" value="61657622" onclick="configDescrLinks(event,this)"><label for="chk_7" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq AJ890224.1</span></label></td>
# Need this!
<td class="ellipsis c2"><span><a title="Go to alignment for Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment" class="deflnDesc" hsp="1" len="2761" ind="7" accs="AJ890224.1" seqfsta="gi|61657622" gi="61657622" seqid="61657622" id="deflnDesc_7" onclick="DisplayAlignFromDescription(this);" href="#alnHdr_61657622">Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment</a></span></td>
<td class="ellipsis c3"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="sciName" title="Taxonomy for Sri Lankan cassava mosaic virus" target="lnk<@rid@>">Sri Lankan cassava mosaic virus</a></span></td>
<td class="ellipsis c4 ui-ncbigrid-column-hidden"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="cmnName" title="Taxonomy for NA">NA</a></span></td>
<td class="c5 ui-ncbigrid-column-hidden"><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="txid" title="Taxonomy for taxid 161378">161378</a></td>
<td class="c6">4102</td>
<td class="c7">4102</td>
<td class="c8">100%</td>
<td class="c9">0.0</td>
# Need this! (Percentage identity)
<td class="c10">93.55%</td>
# Need this!
<td class="c11 acclen">2756</td>
<td class="c12 l lim">
# Need this! (Link and the accession number)
<a href="https://www.ncbi.nlm.nih.gov/nucleotide/AJ890224.1?report=genbank&amp;log$=nucltop&amp;blast_rank=7&amp;RID=H5G050J9016" class="dflSeq" target="lnkH5G050J9016" title="Show report for AJ890224.1">AJ890224.1</a>
</td>
</tr>

"""

'\n\n<tr id="dtr_61657622" ind="7" class="odd dflLnk">\n<td class="l c0"><span class="ind">7</span><input type="checkbox" id="chk_7" class="cb" name="getSeqGi" value="61657622" onclick="configDescrLinks(event,this)"><label for="chk_7" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq AJ890224.1</span></label></td>\n<td class="ellipsis c2"><span><a title="Go to alignment for Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment" class="deflnDesc" hsp="1" len="2761" ind="7" accs="AJ890224.1" seqfsta="gi|61657622" gi="61657622" seqid="61657622" id="deflnDesc_7" onclick="DisplayAlignFromDescription(this);" href="#alnHdr_61657622">Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment</a></span></td>\n<td class="ellipsis c3"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class

In [48]:
res_table = contents["NC_003861"].find("table", attrs = {"id": "dscTable"})
trows = res_table.find_all("tr", attrs = {"class": ["odd dflLnk", "even dflLnk"]})

trows[0]

<tr class="odd dflLnk" id="dtr_20806048" ind="1">
<td class="l c0"><span class="ind">1</span><input class="cb" id="chk_1" name="getSeqGi" onclick="configDescrLinks(event,this)" type="checkbox" value="20806048"/><label for="chk_1" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq NC_003861.1</span></label></td>
<td class="ellipsis c2"><span><a accs="NC_003861.1" class="deflnDesc" gi="20806048" href="#alnHdr_20806048" hsp="1" id="deflnDesc_1" ind="1" len="2755" onclick="DisplayAlignFromDescription(this);" seqfsta="gi|20806048" seqid="20806048" stat="disp" title="Go to alignment for Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col &gt;emb|AJ314737.1| Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col">Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col</a></span></td>
<td class="ellipsis c3"><span><a class="sciName" href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax

In [57]:
# Design the logic to extract the needed details from each table row.

# Extract the name
trows[0].find("a", attrs = {"class": "deflnDesc"}).text

# Extract the percentage identity and sequence length
trows[0].find_all("td", attrs = {"class": ["c10", "c11 acclen"]})

# Extract the accession id 
trows[0].find("a", attrs = {"class": "dflSeq"}).text

# Extract the genbank link
trows[0].find("a", attrs = {"class": "dflSeq"}).get("href")

'https://www.ncbi.nlm.nih.gov/nucleotide/NC_003861.1?report=genbank&log$=nucltop&blast_rank=1&RID=H5G050J9016'

In [71]:
def parse_blast_report(file_path: str) -> dict[str: list[str]]:

    """
    
    """
    
    assert (file_path.endswith(".html") or file_path.endswith(".htm")), "Only HTML files are supported!"
    with open(file_path, 'r') as fp:
        soup = BeautifulSoup(fp.read(), "html.parser")

    table = soup.find("table", attrs = {"id": "dscTable"})
    assert table, "There isn't a <table> tag with id=\"dscTable\" in the HTML file!"

    trows = res_table.find_all("tr", attrs = {"class": ["odd dflLnk", "even dflLnk"]})
    assert trows, "There isn't a <tr> tag with class=\"odd dflLnk\" or class=\"even dflLnk\" in the HTML file!"

    result = {
        "Name": [],
        "PercntIdentity": [],
        "SeqLength": [],
        "Accession": [],
        "GenBankLink": []
    }

    for trow in trows:
        # Extract the name
        result["Name"].append(trow.find("a", attrs = {"class": "deflnDesc"}).text)

        # Extract the percentage identity and sequence length
        prcntident_and_len = trow.find_all("td", attrs = {"class": ["c10", "c11 acclen"]})
        result["PercntIdentity"].append(prcntident_and_len[0].text)
        result["SeqLength"].append(prcntident_and_len[1].text)
        
        # Extract the accession id 
        result["Accession"].append(trow.find("a", attrs = {"class": "dflSeq"}).text)
        
        # Extract the genbank link
        result["GenBankLink"].append(trow.find("a", attrs = {"class": "dflSeq"}).get("href"))

    return result

In [83]:
df = pd.DataFrame()

root = "./data/BLAST/"

for file in os.listdir(root):
    temp = pd.DataFrame(parse_blast_report(root + file))
    df = pd.concat([df, temp], axis = 0)

In [85]:
df.shape

(600, 5)

In [88]:
# Well, if that isn't disappointing

df.drop_duplicates().shape

(100, 5)

In [90]:
df.to_csv("./data/BLAST/results.csv", sep = ',', index = None)

In [33]:
# BLAST results

blast = pd.read_csv("./data/BLAST/results.csv")
blast.shape

(100, 6)

In [34]:
# FASTA query format => https://www.ncbi.nlm.nih.gov/nuccore/NC_003861.1?report=fasta

blast.Accession

0     NC_003861.1
1      KP455484.1
2      OM857615.1
3      AJ890226.1
4      KU550961.1
         ...     
95     MW816856.1
96     JN135236.3
97     MW816855.1
98    NC_038956.1
99     OK236813.1
Name: Accession, Length: 100, dtype: object

In [35]:
os.listdir("./data/BLAST/")

['blastseqs',
 'NC_003861.html',
 'OK362288.html',
 'OK377341.html',
 'OK377342.html',
 'OK377343.html',
 'OK424595.html',
 'results.csv']

In [21]:
URL_TEMPLATE: str = "https://www.ncbi.nlm.nih.gov/nuccore/{}?report=fasta"

In [36]:
# Yes, Edge!

service = webdriver.EdgeService()
options = webdriver.EdgeOptions()
options.add_argument("--headless")

wdriver = webdriver.Edge(options, service)

In [37]:
WRITE_PATH_TEMPLATE = "./data/BLAST/blastseqs/{}.fasta"
URL_TEMPLATE = "https://www.ncbi.nlm.nih.gov/nuccore/{}?report=fasta"

for accession in blast.Accession:
    
    wdriver.get(URL_TEMPLATE.format(accession))
    
    # Wait until the <div id="viewercontent1" class="seq gbff" ....> tag is loaded.
    # 10 seconds wait
    time.sleep(5)
    # _ = WebDriverWait(wdriver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "seq gbff")))
    
    response = wdriver.page_source
    assert response     # Make sure it's not empty.

    soup = BeautifulSoup(response, "html.parser")
    div = soup.find("div", attrs = {"class": "seq gbff", "id": "viewercontent1"})
    
    fasta = div.find("pre").text

    # remove the trailing digits after the period in file names.
    with open(WRITE_PATH_TEMPLATE.format(accession.split('.')[0]), 'w') as fp:
        fp.write(fasta)

wdriver.quit()